In [1]:
import fastai
from fastai.vision import *

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

In [3]:
path = untar_data(URLs.MNIST)

### No Acc

In [8]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing')
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.313806,0.187289,0.943500,00:13


In [9]:
data.batch_size

64

### Naive Acc 

In [10]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,3.557221,4719.024902,0.127200,02:31


### Acc + BnFreeze

In [11]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32), BnFreeze])
learn.loss_func = CrossEntropyFlat(reduction='sum')
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,3.557221,4719.024902,0.127200,02:31


### Increase Momentum 

In [12]:
def set_bn_mom(m:nn.Module, mom=0.9):
    "Set bn layers in eval mode for all recursive children of `m`."
    for l in m.children():
        if isinstance(l, bn_types):
            l.momentum = mom
        set_bn_mom(l, mom)

In [13]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
set_bn_mom(learn.model, mom=0.9)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,3.557221,208308.359375,0.091200,02:31


### Decrease Momentum

In [14]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')
set_bn_mom(learn.model, mom=0.01)
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,3.557221,446.372955,0.151100,02:30


### InstanceNorm

In [15]:
def bn2instance(bn):
    if isinstance(bn, nn.BatchNorm1d): inst = nn.InstanceNorm1d(bn.num_features, affine=True)
    elif isinstance(bn, nn.BatchNorm2d): inst = nn.InstanceNorm2d(bn.num_features, affine=True)
    elif isinstance(bn, nn.BatchNorm3d): inst = nn.InstanceNorm3d(bn.num_features, affine=True)
    
    inst.weight = bn.weight
    inst.bias = bn.bias
    inst.running_mean = nn.Parameter(bn.running_mean, requires_grad=False)
    inst.running_var = nn.Parameter(bn.running_var, requires_grad=False)
    inst.momentum = bn.momentum
    inst.eps = bn.eps
    inst.track_running_stats = bn.track_running_stats
    return (inst).to(bn.weight.device)

In [16]:
def convert_bn(list_mods, func=bn2instance):
    for i in range(len(list_mods)):
        if isinstance(list_mods[i], bn_types):
            list_mods[i] = func(list_mods[i])
        elif list_mods[i].__class__.__name__ in ("Sequential", "BasicBlock"):
            list_mods[i] = nn.Sequential(*convert_bn(list(list_mods[i].children()), func))
    return list_mods

In [17]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [18]:
learn.model = nn.Sequential(*convert_bn(list(learn.model.children()), bn2instance))

In [19]:
learn.freeze()

In [20]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,4.612579,nan,0.098000,04:05


### GroupNorm

In [21]:
def bn2group(bn):
    groupnorm = nn.GroupNorm(4, bn.num_features, affine=True)
    groupnorm.weight = bn.weight
    groupnorm.bias = bn.bias
    groupnorm.eps = bn.eps
    return (groupnorm).to(bn.weight.device)

In [22]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.vgg16_bn, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [23]:
learn.model = nn.Sequential(*convert_bn(list(learn.model.children()), bn2group))

In [24]:
gnorm = nn.GroupNorm(2,5)

In [25]:
learn.freeze()

In [26]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.258457,0.236808,0.964500,02:48


### Resnet + GroupNorm

In [27]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.resnet18, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)])
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [28]:
def change_all_BN(module):
    for i in range(5):
        atr = 'bn'+str(i)
        if hasattr(module, atr):
            setattr(module, atr, bn2group(getattr(module,atr)))

def wrap_BN(model):
    for i in range(len(model)):
        for j in range(len(model[i])):
            if isinstance(model[i][j], bn_types):
                model[i][j] = bn2group(model[i][j])
            elif model[i][j].__class__.__name__ == "Sequential":
                for k in range(len(model[i][j])):
                    if isinstance(model[i][j][k], bn_types):
                        model[i][j][k] = bn2group(model[i][j][k])
                    elif model[i][j][k].__class__.__name__ == "BasicBlock":
                        change_all_BN(model[i][j][k])
                        if hasattr(model[i][j][k],'downsample'):
                            if model[i][j][k].downsample is not None:
                                for l in range(len(model[i][j][k].downsample)):
                                     if isinstance(model[i][j][k].downsample[l], bn_types):
                                        model[i][j][k].downsample[l] = bn2group(model[i][j][k].downsample[l])
    

In [29]:
wrap_BN(learn.model)

In [30]:
learn.freeze()

In [31]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.504959,0.306122,0.949300,03:51


### Resnet + GroupNorm (No Acc)

In [32]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=2)
learn = create_cnn(data, models.resnet18, metrics=accuracy)

In [33]:
wrap_BN(learn.model)
learn.freeze()
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.174799,0.130026,0.961200,05:07


### Resnet + GroupNorm (No Acc) bs = 1

In [34]:
seed_everything(2)
data = ImageDataBunch.from_folder(path, train='training', valid='testing', bs=1)
learn = create_cnn(data, models.resnet18, metrics=accuracy)

In [35]:
wrap_BN(learn.model)
learn.freeze()
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
1,0.060844,0.134206,0.966400,10:51
